In [1]:
# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
# required imports to train a simple TF2 + Keras model for segmentation and package it as CZMODEL
# the CZMODEL can be then imported in ZEN and used for segmentation and image analysis workflows

# general imports
import os
import tensorflow as tf
#from tensorflow_examples.models.pix2pix import pix2pix
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# those functions are provided by the PyPi package called czmodel (by ZEISS)
from czmodel.util.preprocessing import PerImageStandardization, add_preprocessing_layers
from czmodel.model_metadata import ModelMetadata, ModelSpec
from czmodel import convert_from_model_spec, convert_from_json_spec

In [3]:
# Optional: suppress TF warnings
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
print(tf.version.GIT_VERSION, tf.__version__)

v2.2.2-246-gcfe0c80169a 2.2.3


In [5]:
# define the number of channels
# this means using a grayscale image with one channel only
h5file = r"C:\Users\m1srh\OneDrive - Carl Zeiss AG\Trained_Networks\celegans_2CH_apeer.h5"
CHANNELS = 2
NUM_CLASSES = 2

## Create a CZModel from the trained Keras model

In this section we export the trained model to the CZModel format using the czmodel library and some additional meta data all possible parameter choices are described in the [ANN model specification](https://pypi.org/project/czmodel/) that can be found on the PyPi packager for `czmodel`.

### Define Meta-Data

We first define the meta-data needed to run the model within the Intellesis infrastructure. The `czmodel` package offers a named tuple `ModelMetadata` that allows to either parse as JSON file as described or to directly specify the parameters as shown below.

### Create a Model Specification Object

The export functions provided by the `czmodel` package expect a `ModelSpec` tuple that features the Keras model to be exported and the corresponding model metadata.

Therefore, we wrap our model and the `model_metadata` instance into a `ModelSpec` object.

In [6]:
from tensorflow import keras

model = keras.models.load_model(h5file)

ValueError: Unknown layer: FixedDropout

In [ ]:
# Define the model metadata
model_metadata = ModelMetadata.from_params(name='Simple_Nuclei_SegmentationModel', 
                                           color_handling='ConvertToMonochrome',
                                           pixel_type='Gray16',
                                           classes=["Background", "worms"],
                                           border_size=128)


# Create a model specification object used for conversion
model_spec = ModelSpec(model=model, model_metadata=model_metadata)

### Perform model export into *.czmodel file format

The `czmodel` library offers two functions to perform the actual export. 

* `convert_from_json_spec` allows to provide a JSON file with all information to convert a model in SavedModel format on disk to a `.czmodel` file that can be loaded with ZEN.
* `convert_from_model_spec` expects a `ModelSpec` object, an output path and name and optionally target spatial dimensions for the expected input of the exported model. From this information it creates a `.czmodel` file containing the specified model.

```python
convert_from_model_spec(model_spec=model_spec, 
                        output_path=folder_to_store_czmodel, 
                        output_name=name_of_the_model, 
                        spatial_dims=spatial_dims)
```

In [ ]:
# Define dimensions - ZEN Intellesis requires fully defined spatial dimensions.
# This is the tile size used by the ZEN TilingClient to pass a. image to the segmentation service.

# Important: The tile size has to be chosen s.t. inference is possible with the minimum hardware requirements of Intellesis
spatial_dims = 1024, 1024  # Optional: Define target spatial dimensions of the model for inference.

convert_from_model_spec(model_spec=model_spec, 
                        output_path='./czmodel_output', 
                        output_name='simple_nuclei_segmodel', 
                        spatial_dims=spatial_dims)

# In the example above there will be a ""./czmodel_output/simple_nuclei_segmodel.czmodel" file saved on disk.

## Remarks
The generated .czmodel file can be directly loaded into ZEN Intellesis to perform segmentation tasks with the trained model.
If there is already a trained model in SavedModel format present on disk, it can also be converted by providing a meta-data JSON file as described in the [ANN Model Specification](https://pypi.org/project/czmodel/).

The following JSON document describes the same meta-data applied in the use case above:

```json
{
"BorderSize": 8,
"ColorHandling": "ConvertToMonochrome",
"PixelType": "Gray16",
"Classes": ["Background", "Nuclei"],
"ModelPath": "saved_tf2_model_output",
}
```

This information can be copied to a file e.g. in the current working directory `./model_spec.json` that also contains the trained model in SavedModel format e.g. generated by the following line:

In [ ]:
# save the trained TF2.SavedModel as a folder structure
# The folder + the JSON file can be also used to import the model in ZEN

model.save('./saved_tf2_model_output_dims_unset/')
add_preprocessing_layers(model, layers=None, spatial_dims=spatial_dims).save('./saved_tf2_model_output/')

The CZMODEL file (which is essentially a zip file) contains:

* **model guid file**: modelid=e47aabbd-8269-439c-b142-78feec2ed2dd


* **model file**: modelid=e47aabbd-8269-439c-b142-78feec2ed2dd.model


* **model description**: e47aabbd-8269-439c-b142-78feec2ed2dd.xml

**Example of a model XML description**

<img src="./mdpics/model_xml.png" >

To import the newly created model just use the **`Import`** function of the Intellesis Trainable Segmentation module in ZEN.

<img src="./mdpics/zen_import_model1.png" >

Select the **`simple_nuclei_segmodel.czmodel`** file and press the **`Open`** button.

<img src="./mdpics/zen_import_model2.png" >

Use the IP-function **`Segmentation`** to segment an image using the imported CZMODEL (containing the trained network).

<img src="./mdpics/zen_import_model_IPseg.png">

To use the trained model to analyse an image there are two main options

1. directly create an Image Analysis Setting based on the model (no class hierarchy, but very simple)
2. assign the trained model to s specific class inside a customized image analysis setting (shown below)

The crucial step (when not using option 1) is the Select the correct **`Class Segmentation Method`** inside the Image Analysis Wizard.

<img src="./mdpics/zen_import_model_IA1.png">

Use the **`Select Model`** function to assign the trained model and the actual **class** (from the trained model) of interest to assign the model / class to the respective object inside the image analysis setting.

<img src="./mdpics/zen_import_model_IA2.png" >

Now the trained model will be used to segment the image. The built-in ZEN Tiling Client automatically  to chunk the image and deal with complex dimensions, like Use the **`Scenes`** etc.

Additional Post-Processing options, incl. a Minimum Confidence Threshold can be applied to further refine the results.

<img src="./mdpics/zen_import_model_IA3.png" >

Finally, the model can be loaded into ZEN by using the **Import** function on the **JSON file**. 

If the model is supposed to be provided to other parties it is usually easier to exchange .czmodel files instead of SavedModel directories with corresponding JSON meta-data files.

The `czmodel` library also provides a `convert_from_json_spec` function that accepts the above mentioned JSON file and creates a CZModel:

In [ ]:
# This is an additional way how to create a CZMODEL from a saved TF2 model on disk + JSON file.
# The currently recommended way to to create the CZMODEL directly by using czmodel.convert_from_model_spec
# the path to the TF2.SavedModel folder is defined in the JSON shown above

convert_from_json_spec(model_spec_path='model_spec_dims_unset.json',
                       output_path='model_from_json',
                       output_name = 'simple_nuclei_segmodel_from_json',
                       spatial_dims=spatial_dims)

* the path to the saved model folder is defined in the JSON shown above

* **Remark: Due a TF 2.1 bug reloading a model does currently not work correctly.** See issue: https://github.com/tensorflow/tensorflow/issues/37158. This works with TF 2.0 and will be fixed again with TF 2.2. We currently do not have any information if there will be released a patch for TF 2.1 that fixes the issue there.

Use the commands below from a terminal to present the notebook as a slideshow.

`
jupyter nbconvert train_simple_TF2_segmentation_model.ipynb --to slides --post serve 
    --SlidesExporter.reveal_theme=serif 
    --SlidesExporter.reveal_scroll=True 
    --SlidesExporter.reveal_transition=none
`

Or install the [RISE Extension](https://rise.readthedocs.io/en/stable/) to display a a slideshow directly from within the notebook